In [1]:
# imports

import numpy as np
import pandas as pd
import os
import tensorflow as tf

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow import keras

Import and Preprocess

In [2]:
df1=pd.read_csv('Dados_Emotiv/7_AÇÃO_abrir_mão_PENSAMENTO.csv')
df2=pd.read_csv('Dados_Emotiv/8_AÇÃO_abrir_mão_PENSAMENTOORIENTADO.csv')
df3=pd.read_csv('Dados_Emotiv/9_AÇÃO_fechar_mão_PENSAMENTO.csv')
df4=pd.read_csv('Dados_Emotiv/10_AÇÃO_fechar_mão_PENSAMENTOORIENTADO.csv')

In [3]:
#concatenating the datasets and reseting the indexes

df_open=pd.concat([df1, df2], ignore_index=True)

df_close=pd.concat([df3, df4], ignore_index=True)

In [4]:
#Action "open hand" = 1

df_open['Action']=1


#Action "close hand" = 2

df_close['Action']=2


In [5]:
df=pd.concat([df_open, df_close], ignore_index=True)

In [8]:
print(df)

#df.to_csv('df_emotiv.csv') 

        Unnamed: 0          AF3           T7           Pz           T8  \
0                0  4272.179487  4206.025641  4139.487179  4076.794871   
1                1  4268.589744  4207.692308  4139.487179  4075.000000   
2                2  4272.051282  4235.128205  4143.205128  4102.820513   
3                3  4269.487180  4208.076923  4140.256410  4102.948718   
4                4  4268.846154  4207.820513  4135.512820  4075.897436   
...            ...          ...          ...          ...          ...   
153595      153595  4268.974359  4301.666667  4143.333333  4107.820513   
153596      153596  4239.487179  4299.615385  4139.743590  4105.512820   
153597      153597  4266.666667  4241.794872  4138.205128  4106.153846   
153598      153598  4299.487180  4241.538462  4140.769231  4108.589743   
153599      153599  4299.871795  4270.769231  4141.794872  4109.615384   

                AF4  Action  
0       4205.384615       1  
1       4203.974359       1  
2       4205.641026  

In [7]:
df=pd.read_csv('df_emotiv.csv')

In [9]:
df.columns

column_names=['AF3', ' T7', ' Pz', ' T8', ' AF4']
all_columns=['AF3', ' T7', ' Pz', ' T8', ' AF4', 'Action']

Data Normalization

In [4]:
df[column_names] = preprocessing.normalize(df[column_names], axis=0)
df = pd.DataFrame(df, columns=all_columns)
df.head()

,AF3,T7,Pz,T8,AF4,Action
0,0.002560,0.002532,0.002571,0.002550,0.002543,1
1,0.002558,0.002533,0.002571,0.002548,0.002542,1
2,0.002560,0.002549,0.002573,0.002566,0.002543,1
3,0.002558,0.002533,0.002572,0.002566,0.002544,1
4,0.002558,0.002533,0.002569,0.002549,0.002543,1


In [16]:
#df.to_csv('df_emotiv_normalized.csv')

df=pd.read_csv('df_emotiv_normalized.csv')

print(df)

        Unnamed: 0       AF3        T7        Pz        T8       AF4  Action
0                0  0.002560  0.002532  0.002571  0.002550  0.002543       1
1                1  0.002558  0.002533  0.002571  0.002548  0.002542       1
2                2  0.002560  0.002549  0.002573  0.002566  0.002543       1
3                3  0.002558  0.002533  0.002572  0.002566  0.002544       1
4                4  0.002558  0.002533  0.002569  0.002549  0.002543       1
...            ...       ...       ...       ...       ...       ...     ...
153595      153595  0.002558  0.002590  0.002574  0.002569  0.002622       2
153596      153596  0.002540  0.002588  0.002571  0.002567  0.002621       2
153597      153597  0.002556  0.002553  0.002570  0.002568  0.002620       2
153598      153598  0.002576  0.002553  0.002572  0.002569  0.002620       2
153599      153599  0.002576  0.002571  0.002573  0.002570  0.002622       2

[153600 rows x 7 columns]


Creating training and testing sets

In [18]:
x=df[['AF3', ' T7', ' Pz', ' T8', ' AF4']]
y=df['Action']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

SVM

In [20]:
modelSVM=svm.SVC()
modelSVM.fit(x_train,y_train)
predictSVM = modelSVM.predict(x_test)

SVM Scores

Accuracy with normalized data: SC = 69.51171875

Accuracy without normalized data: SC1 = 68.095703125

In [21]:
#SC1=modelSVM.score(x_test,y_test)*100

SC=modelSVM.score(x_test,y_test)*100

SC

69.51171875

In [22]:
print(classification_report(y_test,predictSVM))

              precision    recall  f1-score   support

           1       0.74      0.59      0.66     15174
           2       0.66      0.80      0.73     15546

    accuracy                           0.70     30720
   macro avg       0.70      0.69      0.69     30720
weighted avg       0.70      0.70      0.69     30720



In [23]:
print("\nMatriz de Confusão\r\n",
pd.crosstab(y_test,predictSVM,rownames=['Real'],colnames=['Predito'], margins=True, margins_name='Todos'))
dic_SVM=metrics.classification_report(y_test,predictSVM,target_names=['1','2'], output_dict=True)


Matriz de Confusão
 Predito      1      2  Todos
Real                        
1         8898   6276  15174
2         3090  12456  15546
Todos    11988  18732  30720


ANN

In [24]:
#Inicializing weight and random seed

np.random.seed(42)
tf.random.set_seed(42)

In [37]:
#Creating layers

modelANN = keras.models.Sequential()

modelANN.add(keras.layers.Flatten(input_shape=[5,]))
modelANN.add(keras.layers.Dense(300, activation="relu"))
modelANN.add(keras.layers.Dense(100, activation="relu"))
modelANN.add(keras.layers.Dense(10, activation="softmax"))

modelANN.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 5)                 0         
                                                                 
 dense_12 (Dense)            (None, 300)               1800      
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 dense_14 (Dense)            (None, 10)                1010      
                                                                 
Total params: 32,910
Trainable params: 32,910
Non-trainable params: 0
_________________________________________________________________


In [39]:
#Compiling the model with desired loss function

modelANN.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])

In [40]:
#Training the model

model_history = modelANN.fit(x_train,y_train,epochs=300,validation_data=(x_test,y_test))

Epoch 1/300
3840/3840 [==============================] - 5s 1ms/step - loss: 0.7667 - accuracy: 0.4992 - val_loss: 0.6944 - val_accuracy: 0.5061
Epoch 2/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6949 - accuracy: 0.5001 - val_loss: 0.6949 - val_accuracy: 0.4939
Epoch 3/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6945 - accuracy: 0.5013 - val_loss: 0.6952 - val_accuracy: 0.4939
Epoch 4/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6944 - accuracy: 0.5020 - val_loss: 0.6937 - val_accuracy: 0.5061
Epoch 5/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6944 - accuracy: 0.5011 - val_loss: 0.6933 - val_accuracy: 0.5061
Epoch 6/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6942 - accuracy: 0.5028 - val_loss: 0.6950 - val_accuracy: 0.4939
Epoch 7/300
3840/3840 [==============================] - 4s 1ms/step - loss: 0.6942 - accuracy: 0.5013 - val_loss: 0.6938 - val_ac

ANN Score

In [41]:
modelANN.evaluate(x_test,y_test)*100

960/960 [==============================] - 1s 866us/step - loss: 0.6936 - accuracy: 0.4939


[0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.6935684680938721,
 0.49394530057907104,
 0.69356846